<a href="https://colab.research.google.com/github/GianEscher/quantum-lab-finance/blob/main/markowitz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-bokeh





In [2]:
import pandas as pd
import pandas_bokeh
import yfinance as yfin
import numpy as np
from pandas_datareader import data as pdr
yfin.pdr_override()

import matplotlib.pyplot as plt

COMENTÁRIO TESTE

In [8]:

#data_inicial = '2023-05-10'
#data_final = '2023-05-16'     #stops 2 days before this dude

stocks_registry = []
gain = []
volatility = []
risks = []

def setup():

  n_stocks = input('insert the number of stocks, my dude: ')
  data_inicial = input('insert the start data, as "yy-mm-dd": ')
  data_final = input('insert the ending data, as "yy-mm-dd": ')

  i=0
  while(i<int(n_stocks)):

    stocks_raw = []
    stocks_gain = []

    #apparently, if i append a global list inside another global list, any alterations made to the former will be DINAMICALLY updated in the latter too
    #search for that later. seems like quite the phantasmagorical behaviour

    stocks = input('insert the stock code: ')
    history = pdr.DataReader(stocks, data_inicial, data_final)

    stocks_raw = history['Open'].tolist()
    
    j=1
    while(j<len(stocks_raw)):
      stocks_gain.append((stocks_raw[j]-stocks_raw[j-1])/stocks_raw[j-1])
      j+=1

    stocks_registry.append(stocks_gain)

    sum = 0
    for a in stocks_gain:
      sum += a
    gain.append(sum/len(stocks_gain)) #calculate the mean value of my gains
    
    sd = 0
    for a in stocks_gain:
      sd += (a-sum)*(a-sum)
    sd == np.sqrt(sd/len(stocks_gain) )   
    volatility.append(sd)

    #stocks_raw.clear()
    #stocks_gain.clear()
    i+=1

  print(stocks_registry)
  print(gain)
  print(volatility)
  
  
  #risk matrix building
  i=0
  while(i<len(volatility)):
    j=0
    row = []

    while(j<len(volatility)):
      if(i==j):
        row.append(volatility[i]*volatility[i])
        
      else:
        row.append(covar(stocks_registry[i],gain[i],stocks_registry[j],gain[j]))
      j+=1

    risks.append(row)
    i+=1

  print(risks)

  
  

def covar(list_A, mean_A, list_B, mean_B):
  sum = 0
  i=0
  lenght=len(list_A)

  while(i<lenght):
    sum+=(list_A[i]-mean_A)*(list_B[i]-mean_B)
    i+=1
    
  return sum/lenght

setup()

insert the number of stocks, my dude: 3
insert the start data, as "yy-mm-dd": 2023-05-10
insert the ending data, as "yy-mm-dd": 2023-05-18
insert the stock code: GOOG
[*********************100%***********************]  1 of 1 completed
insert the stock code: IBM
[*********************100%***********************]  1 of 1 completed
insert the stock code: AMZN
[*********************100%***********************]  1 of 1 completed
[[0.06734221421539956, 0.00983945609911022, -0.004358992617354434, 0.0029187395786784845, 0.028674128405522525], [0.00024591179459135395, -0.004999123076042907, 0.013096090024967555, 0.0028455160497649896, 0.0047831696358629425], [0.02710453604564312, 0.010177473612862954, -0.009005011619589259, -0.0008996713697553233, 0.034578984532768475]]
[0.02088310913627127, 0.0031943128858287874, 0.012391262240385992]
[0.03818933274874801, 0.0009928075623799977, 0.013631596150801202]
[[0.001458425135794597, -5.555830988736158e-05, 0.00033194579685141964], [-5.555830988736158e